# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

[In this notebook you're provided with hints and brief instructions and thinking prompts. Don't ignore them as they are designed to equip you with the structure for the project and will help you analyze what you're doing on a deeper level. Before submitting your project, make sure you remove all hints and descriptions provided to you. Instead, make this report look as if you're sending it to your teammates to demonstrate your findings - they shouldn't know you had some external help from us! To help you out, we've placed the hints you should remove in square brackets.]

[Before you dive into analyzing your data, explain the purposes of the project and hypotheses you're going to test.]

## Open the data file and have a look at the general information. 

[Start with importing the libraries and loading the data. You may realise that you need additional libraries as you go, which is totally fine - just make sure to update this section when you do.]

In [1]:
# Loading all the libraries
import numpy as np
import pandas as pd 

# Load the data
# df = pd.read_csv ('/datasets/credit_scoring_eng.csv')
df = pd.read_csv('/content/credit_scoring_eng.csv')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

[Now let's explore our data. You'll want to see how many columns and rows it has, look at a few rows to check for potential issues with the data.]

In [2]:
# Let's see how many rows and columns our dataset has
row , col = df.shape
print( ' we have ' + str(row) +' rows' + ' and '  +str(col)+ ' columns'  )

 we have 21525 rows and 12 columns


In [3]:
# let's print the first N rows
N  = 5

print(df.head(N))


   children  days_employed  ...  total_income                  purpose
0         1   -8437.673028  ...     40620.102    purchase of the house
1         1   -4024.803754  ...     17932.802             car purchase
2         0   -5623.422610  ...     23341.752    purchase of the house
3         3   -4124.747207  ...     42820.568  supplementary education
4         0  340266.072047  ...     25378.572        to have a wedding

[5 rows x 12 columns]


[Describe what you see and notice in your printed data sample. Are there any issues that may need further investigation and changes?]

well there is inconistent in the use of upper and lower charcther in string, 
I would have replace the cols of : education , income_type , family status to categories after using lower charcter than compare to the id column to make sure there is no inconsistents .
check that gender type is all capital/lower charcter
check that numeric cols (childern , debt , total income are all numeric/boolean)

and make sure that ther are no missing values

In [4]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[Are there missing values across all columns or just a few? Briefly describe what you see in 1-2 sentences.]
yes the days employed and total income are both having missing values- I assume that they wern't filled with 0 for unemployed 

In [5]:
# Let's look at the filtered table with missing values in the the first column with missing data

# print( df.groupby(['days_employed'])['total_income'].mean() )
df.index[df.isnull().any(axis=1)]

Int64Index([   12,    26,    29,    41,    55,    65,    67,    72,    82,
               83,
            ...
            21415, 21423, 21426, 21432, 21463, 21489, 21495, 21497, 21502,
            21510],
           dtype='int64', length=2174)

[Do missing values seem symmetric? Can we be sure in this assumption? Explain your thoughts briefly in this section. You may probably want to conduct further investigations, and count the missing values in all the rows with missing values to confirm the the missing samples are of the same size.]
no

In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

sum(df.index[df.isnull().any(axis=1)]) # all
where_both = df.total_income.isnull() == df.days_employed.isnull()
both = sum( where_both.tolist())
only_days = where_both != df.days_employed.isnull().to_list()
days = sum( only_days.tolist())
days
only_income = where_both == df.total_income.isnull().to_list()
print(sum(only_income))


2174


**Intermediate conclusion**

[Does the number of rows in the filtered table match the number of missing values? What conclusion can we make from this?] - Yes, the number of rows in the filtered table match the number of missing values.


[Calculate the percentage of the missing values compared to the whole dataset. Is it a considerably large piece of data? If so, you may want to fill the missing values. To do that, firstly we should consider whether the missing data could be due to the specific client characteristic, such as employment type or something else. You will need to decide which characteristic *you* think might be the reason. Secondly, we should check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic.] - The percentage of the missing values is approx 10% of the whole dataset. Yes, it is a large piece of data and we cannot ignore it. So, we need to fill that values.

[Explain your next steps and how they correlate with the conclusions you made so far.]

In [7]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
print(df.isnull().sum())

column_with_missing_values = []
for col in df.columns:
    if df[col].isnull().sum()>0:
        column_with_missing_values.append(col)
        
print('\n\nColumns with missing values : ',column_with_missing_values)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Columns with missing values :  ['days_employed', 'total_income']


In [8]:
# Checking distribution

print("Percentage of missing values compared to the whole dataset :\n")
for col in column_with_missing_values:
    print('Column : {}\t Missing value percentage : {}'.format(col,(df[col].isnull().sum()/len(df))*100))


Percentage of missing values compared to the whole dataset :

Column : days_employed	 Missing value percentage : 10.099883855981417
Column : total_income	 Missing value percentage : 10.099883855981417


[Describe your findings here.]

**Possible reasons for missing values in data**

[Propose your ideas on why you think the values might be missing. Do you think they are missing randomly or there are any patterns?] - I think there may be some patterns as if days_employed value is missing then total_income value is also missing. It means days_employed and total_income value depends on each other. If we don't have the data in days_employed column then also we don't have data for total_income.

[Let's start checking whether the missing values are random.] 

In [9]:
# Checking the distribution in the whole dataset

df[df.isna().any(axis=1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Intermediate conclusion**

[Is the distribution in the original dataset similar to the distribution of the filtered table? What does that mean for us?]

[If you think we can't make any conclusions yet, let's investigate our dataset further. Let's think about other reasons that could lead to data missing and check if we can find any patterns that may lead us to thinking that the missing values are not random. Because this is your work, this is section is optional.]

In [10]:
# Check for other reasons and patterns that could lead to missing values



**Intermediate conclusion**

[Can we finally confirm that missing values are accidental? Check for anything else that you think might be important here.] - If you see the below dataframe, the missing values are accidental for the column days_employed but not for total_income. As from the dataframe we can conclude that the total_income depends on the days_employed.

In [11]:
# Checking for other patterns - explain which

df[df.isna().any(axis=1)][['days_employed','total_income']]

,days_employed,total_income
12,NaN,NaN
26,NaN,NaN
29,NaN,NaN
41,NaN,NaN
55,NaN,NaN
...,...,...
21489,NaN,NaN
21495,NaN,NaN
21497,NaN,NaN
21502,NaN,NaN


**Conclusions**

[Did you find any patterns? How did you come to this conclusion?] - Yes, there is a patterns between days_employed and total_income.

[Explain how you will address the missing values. Consider the categories in which values are missing.] - We can fill numerical values with mean or median or if we have categorical values then we can use forward fill method to fill missing values. 

[Briefly plan your next steps for transforming data. You will probably need to address different types of issues: duplicates, different registers, incorrect artifacts, and missing values.]  

## Data transformation

[Let's go through each column to see what issues we may have in them.]

[Begin with removing duplicates and fixing educational information if required.]

In [12]:
# Let's see all values in education column to check if and what spellings will need to be fixed
set(df['education'])

{"BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college'}

In [13]:
# Fix the registers if required
df['education'].replace(["BACHELOR'S DEGREE","bachelor's degree"],["Bachelor's Degree","Bachelor's Degree"], inplace = True)
df['education'].replace(["GRADUATE DEGREE","graduate degree"],["Graduate Degree","Graduate Degree"], inplace = True)
df['education'].replace(["PRIMARY EDUCATION","primary education"],["Primary Education","Primary Education"], inplace = True)
df['education'].replace(["SECONDARY EDUCATION","secondary education"],["Secondary Education","Secondary Education"], inplace = True)
df['education'].replace(["SOME COLLEGE","some college"],["Some College","Some College"], inplace = True)

In [14]:
# Checking all the values in the column to make sure we fixed them
set(df['education'])

{"Bachelor's Degree",
 'Graduate Degree',
 'Primary Education',
 'Secondary Education',
 'Some College'}

[Check the data the `children` column]

In [15]:
# Let's see the distribution of values in the `children` column
set(df['children'])

{-1, 0, 1, 2, 3, 4, 5, 20}

[Are there any strange things in the column? If yes, how high is the percentage of problematic data? How could they have occurred? Make a decision on what you will do with this data and explain you reasoning.] - Yes, the column has the strange things as it has -1 and 20. Children with -1 and 20 values is about 0.21% and 0.35% respectively.

In [16]:
# [fix the data based on your decision]
print('Children = -1 percentage : ',(len(df[df['children'] == -1])/len(df))*100)
print('Children = 20 percentage : ',(len(df[df['children'] == 20])/len(df))*100)
df = df[df['children'] != -1]
df = df[df['children'] != 20]

Children = -1 percentage :  0.2183507549361208
Children = 20 percentage :  0.3530778164924506


In [17]:
# Checking the `children` column again to make sure it's all fixed
set(df['children'])

{0, 1, 2, 3, 4, 5}

[Check the data in the `days_employed` column. Firstly think about what kind of issues could there be and what you may want to check and how you will do it.]

In [18]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
print(set(df['days_employed']))

# As there are some negative values and missing values
print("\n\nPercentage of negative values to the whole dataset = ",(len(df[df['days_employed']<0])/len(df))*100)
print("\n\nPercentage of missing values to the whole dataset = ",(df['days_employed'].isnull().sum()/len(df))*100)

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 390775.1299182249, 393246.7210757576, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 393280.682279755, 393298.380985794, 393331.5005366179, 393337.57776490937, 393370.2008331024, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 393380.7656047209, 393389.7646601208, 393404.0845203994, 393429.5161267279, 393452.1445002313, 393506.2668509173, 393547.941173224, 390842.2702300489, 390846.8538258114, 393619.9576595304, 393631.7154380664, 393646.3766012972, 393649.355581225, 393651.23696145095, 393693.1682437829, nan, nan, nan, nan, nan, nan, nan, nan, 393711.519922678, nan, 393725.4267949609, 390878.8516730853, 393747.30750915024, 393769.9343824062, 393799.97259229, 393823.69583021046, 393825.2914776166, 393828.1688682389, 393831.9264719042, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 393852.6033737048, 393859.1924682685, 393880.2417671519, 393921.4581466925, 393964.8409971221, 393990.55333276594, nan,

[If the amount of problematic data is high, it could've been due to some technical issues. We may probably want to propose the most obvious reason why it could've happened and what the correct data might've been, as we can't drop these problematic rows.]

In [19]:
# Address the problematic values, if they exist
days_employed = []

for days in df['days_employed']:
    try:
      days_employed.append(int(abs(days)))
    except:
      days_employed.append(days)
df['days_employed'] = days_employed

In [20]:
# Check the result - make sure it's fixed
set(df['days_employed'])

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 390775.0,
 24.0,
 360475.0,
 30.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 34.0,
 37.0,
 33.0,
 39.0,
 46.0,
 47.0,
 50.0,
 51.0,
 54.0,
 55.0,
 58.0,
 60.0,
 61.0,
 62.0,
 393280.0,
 65.0,
 67.0,
 69.0,
 70.0,
 72.0,
 73.0,
 74.0,
 75.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 360531.0,
 87.0,
 88.0,
 86.0,
 90.0,
 91.0,
 92.0,
 89.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 393337.0,
 122.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0,
 139.0,
 140.0,
 141.0,
 142.0,
 143.0,
 144.0,
 145.0,
 146.0,
 147.0,
 148.0,
 149.0,
 150.0,
 151.0,
 152.0,
 153.0,
 154.0,
 155.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

[Let's now look at the client's age and whether there are any issues there. Again, think about what can data can be strange in this column, i.e. what cannot be someone's age.]

In [21]:
# Check the `dob_years` for suspicious values and count the percentage
set(df['dob_years'])


{0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75}

[Decide what you'll do with the problematic values and explain why.]

In [22]:
# Address the issues in the `dob_years` column, if they exist

df['dob_years'].replace([0],[int(df['dob_years'].mean())], inplace = True)

In [23]:
# Check the result - make sure it's fixed
set(df['dob_years'])

{19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75}

[Now let's check the `family_status` column. See what kind of values there are and what problems you may need to address.]

In [24]:
# Let's see the values for the column

set(df['family_status'])

{'civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower'}

In [25]:
# Address the problematic values in `family_status`, if they exist

# there is no problem exist in the family_status column

In [26]:
# Check the result - make sure it's fixed
set(df['family_status'])

{'civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower'}

[Now let's check the `gender` column. See what kind of values there are and what problems you may need to address]

In [27]:
# Let's see the values in the column
set(df['gender'])

{'F', 'M', 'XNA'}

In [28]:
# Address the problematic values, if they exist
df = df[df.gender != 'XNA']

In [29]:
# Check the result - make sure it's fixed

set(df['gender'])

{'F', 'M'}

[Now let's check the `income_type` column. See what kind of values there are and what problems you may need to address]

In [30]:
# Let's see the values in the column
set(df['income_type'])

{'business',
 'civil servant',
 'employee',
 'entrepreneur',
 'paternity / maternity leave',
 'retiree',
 'student',
 'unemployed'}

In [31]:
# Address the problematic values, if they exist

#there is no problem exist in the income_type column

In [32]:
# Check the result - make sure it's fixed

set(df['income_type'])

{'business',
 'civil servant',
 'employee',
 'entrepreneur',
 'paternity / maternity leave',
 'retiree',
 'student',
 'unemployed'}

[Now let's see if we have any duplicates in our data. If we do, you'll need to decide what you will do with them and explain why.]

In [33]:
# Checking duplicates

duplicate = df[df.duplicated()]
print("Duplicate Rows : ",len(duplicate))
duplicate

Duplicate Rows :  71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,Secondary Education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58,Secondary Education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34,Bachelor's Degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60,Secondary Education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58,Secondary Education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,Secondary Education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,NaN,60,Secondary Education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47,Secondary Education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30,Bachelor's Degree,0,married,0,F,employee,0,NaN,buy commercial real estate


In [34]:
# Address the duplicates, if they exist
df.drop_duplicates(inplace = True)


In [35]:
# Last check whether we have any duplicates
duplicate = df[df.duplicated()]
print("Duplicate Rows : ", len(duplicate))


Duplicate Rows :  0


In [36]:
# Check the size of the dataset that you now have after your first manipulations with it
len(df)

21330

[Describe your new dataset: briefly say what's changed and what's the percentage of the changes, if there were any.]


# Working with missing values

[To speed up working with some data, you may want to work with dictionaries for some values, where IDs are provided. Explain why and which dictionaries you will work with.]

In [37]:
# Find the dictionaries
dictionary_education = {}
for id in list(set(df['education_id'])):
    education = list(set(df[df['education_id'] == id]['education']))[0]
    dictionary_education[id] = education

dictionary_family = {}
for id in list(set(df['family_status_id'])):
    family_status = list(set(df[df['family_status_id'] == id]['family_status']))[0]
    dictionary_family[id] = family_status

print(dictionary_education)

print(dictionary_family)

{0: "Bachelor's Degree", 1: 'Secondary Education', 2: 'Some College', 3: 'Primary Education', 4: 'Graduate Degree'}
{0: 'married', 1: 'civil partnership', 2: 'widow / widower', 3: 'divorced', 4: 'unmarried'}


### Restoring missing values in `total_income`

[Briefly state which column(s) have values missing that you need to address. Explain how you will fix them.] - days_employed and total_income are the two columns that have missing values.


[Start with addressing total income missing values. Create and age category for clients. Create a new column with the age category. This strategy can help with calculating values for the total income.] 


In [38]:
# Let's write a function that calculates the age category
def age_cat(age):
    if  age <= 25:
        return 'less than 25'
    if 25 < age <= 35:
        return '25-35'
    if 35 < age <= 45:
        return '35-45'
    if 45 < age <= 55:
        return '45-55'
    if 55 < age <= 65:
        return '55-65'
    if age > 65:
        return 'above 65'
  


    

In [39]:
# Test if the function works
print(age_cat(23))
print(age_cat(43))
print(age_cat(67))

less than 25
35-45
above 65


In [40]:
# Creating new column based on function
df['age category'] = df['dob_years'].apply(lambda x: age_cat(x))


In [41]:
# Checking how values in the new column

df[['dob_years','age category']]

,dob_years,age category
0,42,35-45
1,36,35-45
2,33,25-35
3,32,25-35
4,53,45-55
...,...,...
21520,43,35-45
21521,67,above 65
21522,38,35-45
21523,38,35-45


[Think about the factors on which income usually depends. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. To make this decision you will probably want to look at the distribution of the factors you identified as impacting one's income.]

[Create a table that only has data without missing values. This data will be used to restore the missing values.]

In [42]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
df_without_nan = df.dropna()
df_without_nan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age category
0,1,8437.0,42,Bachelor's Degree,0,married,0,F,employee,0,40620.102,purchase of the house,35-45
1,1,4024.0,36,Secondary Education,1,married,0,F,employee,0,17932.802,car purchase,35-45
2,0,5623.0,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,25-35
3,3,4124.0,32,Secondary Education,1,married,0,M,employee,0,42820.568,supplementary education,25-35
4,0,340266.0,53,Secondary Education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,45-55


In [43]:
# Look at the mean values for income based on your identified factors
income_mean = df_without_nan['total_income'].mean()
income_mean

26791.50317506113

In [44]:
# Look at the median values for income based on your identified factors
df_without_nan['total_income'].median()

23202.87

[Repeat such comparisons for multiple factors. Make sure you consider different aspects and explain your thinking process.]



[Make a decision on what characteristics define income most and whether you will use a median or a mean. Explain why you made this decision]


In [45]:
#  Write a function that we will use for filling in missing values
def fill_missing_value(value):
    if value == 'missing':
        return income_mean
    else:
        return value
    
        

In [46]:
# Check if it works
x = df[df.isna().any(axis=1)]
x.fillna('missing', inplace = True)
x['missing_value'] = x['total_income'].apply(lambda x: fill_missing_value(x))
print(x['missing_value'])

12       26791.503175
26       26791.503175
29       26791.503175
41       26791.503175
55       26791.503175
             ...     
21489    26791.503175
21495    26791.503175
21497    26791.503175
21502    26791.503175
21510    26791.503175
Name: missing_value, Length: 2091, dtype: float64


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [47]:
# Apply it to every row
df['total_income'].fillna('missing', inplace = True)
df['total_income'] = df['total_income'].apply(lambda x: fill_missing_value(x))

In [48]:
# Check if we got any errors
df['total_income'].isnull().sum()



0

[If you've came across errors in preparing the values for missing data, it probably means there's something special about the data for the category. Give it some thought - you may want to fix some things manually, if there's enough data to find medians/means.]


In [49]:
# Replacing missing values if there are any errors
df['total_income'].replace(to_replace = np.nan, value = income_mean, inplace = True) 

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [50]:
# Checking the number of entries in the columns
len(df['total_income'])


21330

###  Restoring values in `days_employed`

[Think about the parameters that may help you restore the missing values in this column. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. You will probably conduct a research similar to the one you've done when restoring data in a previous column.]

In [51]:
# Distribution of `days_employed` medians based on your identified parameters
df['days_employed'].median()

2194.0

[Decide what you will use: means or medians. Explain why.]

In [52]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
def calculate_mean(list_values):
    return sum(list_values)/len(list_values)


In [53]:
# Check that the function works

calculate_mean([2,4,6,8])

5.0

In [54]:
# Apply function to the income_type

calculate_mean(df['total_income'])

26791.503175061636

In [55]:
# Check if function worked

df['total_income'].mean()

26791.503175061636

In [56]:
calculate_mean(df['days_employed'].dropna())

67030.55319923072

In [57]:
# Replacing missing values

df['total_income'].replace(to_replace = np.nan, value = calculate_mean(df['total_income'].dropna()), inplace = True) 
df['days_employed'].replace(to_replace = np.nan, value = calculate_mean(df['days_employed'].dropna()), inplace = True) 

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [58]:
# Check the entries in all columns - make sure we fixed all missing values
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age category        0
dtype: int64

## Categorization of data

[To answer the questions and test the hypotheses, you will want to work with categorized data. Look at the questions that were posed to you and that you should answer. Think about which of the data will need to be categorized to answer these questions. Below you will find a template through which you can work your way when categorizing data. The first step-by-step processing covers the text data; the second one addresses the numerical data that needs to be categorized. You can use both or none of the suggested instructions - it's up to you.]

[Despite of how you decide to address the categorization, make sure to provide clear explanation of why you made your decision. Remember: this is your work and you make all decisions in it.]


In [59]:
# Print the values for your selected data for categorization

df['education']

0          Bachelor's Degree
1        Secondary Education
2        Secondary Education
3        Secondary Education
4        Secondary Education
                ...         
21520    Secondary Education
21521    Secondary Education
21522    Secondary Education
21523    Secondary Education
21524    Secondary Education
Name: education, Length: 21330, dtype: object

[Let's check unique values]

In [60]:
# Check the unique values
set(df['education'])

{"Bachelor's Degree",
 'Graduate Degree',
 'Primary Education',
 'Secondary Education',
 'Some College'}

[What main groups can you identify based on the unique values?]

[Based on these themes, we will probably want to categorize our data.]


In [61]:
# Let's write a function to categorize the data based on common topics
def categorize_data(education):
    if education in ["Bachelor's Degree","Graduate Degree"]:
        return "Graduated"
    if education == "Some College":
        return "Undergraduate"
    if education in ['Primary Education','Secondary Education']:
        return "In school"
        

In [62]:
# Create a column with the categories and count the values for them
df['education_status'] = df['education'].apply(lambda x: categorize_data(x))
from collections import Counter
Counter(df['education_status'])

Counter({'Graduated': 5233, 'In school': 15357, 'Undergraduate': 740})

[If you decide to categorize the numerical data, you'll need to come up with the categories for it too.]

In [63]:
# Looking through all the numerical data in your selected column for categorization
df[['children','days_employed','dob_years','education_id','family_status_id','debt','total_income']]

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
0,1,8437.0,42,0,0,0,40620.102
1,1,4024.0,36,1,0,0,17932.802
2,0,5623.0,33,1,0,0,23341.752
3,3,4124.0,32,1,0,0,42820.568
4,0,340266.0,53,1,1,0,25378.572
...,...,...,...,...,...,...,...
21520,1,4529.0,43,1,1,0,35966.698
21521,0,343937.0,67,1,0,0,24959.969
21522,1,2113.0,38,1,1,1,14347.610
21523,3,3112.0,38,1,0,1,39054.888


In [64]:
# Getting summary statistics for the column

df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21330.000000,21330.000000,21330.000000,21330.000000,21330.000000,21330.000000,21330.000000
mean,0.474027,67030.553199,43.480450,0.816925,0.974496,0.081200,26791.503175
std,0.752389,132137.774132,12.219089,0.549235,1.421820,0.273149,15667.684517
min,0.000000,24.000000,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,1024.000000,33.000000,1.000000,0.000000,0.000000,17200.597250
50%,0.000000,2594.000000,43.000000,1.000000,0.000000,0.000000,24972.716500
75%,1.000000,67030.553199,53.000000,1.000000,1.000000,0.000000,31331.348000
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,362496.645000


[Decide what ranges you will use for grouping and explain why.]

In [65]:
# Creating function for categorizing into different numerical groups based on ranges

def categorize_data(income):
    if income <= 20000:
        return "Less than 20k"
    if 20000 < income <= 30000:
        return "20k to 30k"
    if 30000 < income <= 40000:
        return "30k to 40k"
    if income > 40000:
        return "Above 60k"

In [66]:
# Creating column with categories
df['income_range'] = df['total_income'].apply(lambda x: categorize_data(x))

In [67]:
# Count each categories values to see the distribution
Counter(df['income_range'])

Counter({'20k to 30k': 8110,
         '30k to 40k': 3091,
         'Above 60k': 2796,
         'Less than 20k': 7333})

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [77]:
# Check the children data and paying back on time
paying_back_on_time = df[df['debt'] == 0]
print(paying_back_on_time['children'].head(10))
# Calculating default-rate based on the number of children

for children, number in dict(Counter(paying_back_on_time['children'])).items():
    print("No. of children : {} \t Default-Rate : {}".format(children, round((number/len(paying_back_on_time))*100,2)))



0    1
1    1
2    0
3    3
4    0
5    0
6    0
7    0
8    2
9    0
Name: children, dtype: int64
No. of children : 1 	 Default-Rate : 22.27
No. of children : 0 	 Default-Rate : 66.47
No. of children : 3 	 Default-Rate : 1.55
No. of children : 2 	 Default-Rate : 9.48
No. of children : 4 	 Default-Rate : 0.19
No. of children : 5 	 Default-Rate : 0.05


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
children,1.000000,-0.226063,-0.319666,-0.026994,-0.158966,0.024635,0.030053
days_employed,-0.226063,1.000000,0.577674,0.076422,0.002566,-0.046313,-0.135633
dob_years,-0.319666,0.577674,1.000000,0.064699,-0.068634,-0.071685,-0.053420
education_id,-0.026994,0.076422,0.064699,1.000000,0.007769,0.052842,-0.170876
family_status_id,-0.158966,0.002566,-0.068634,0.007769,1.000000,0.020181,-0.007670
debt,0.024635,-0.046313,-0.071685,0.052842,0.020181,1.000000,-0.011887
total_income,0.030053,-0.135633,-0.053420,-0.170876,-0.007670,-0.011887,1.000000


**Conclusion**

[Write your conclusions based on your manipulations and observations.]


**Is there a correlation between family status and paying back on time?**

In [79]:
# Check the family status data and paying back on time
print(paying_back_on_time['family_status'].head(10))


# Calculating default-rate based on family status

for family, number in dict(Counter(paying_back_on_time['family_status'])).items():
    print("family_status  : {} \t Default-Rate : {}".format(family, round((number/len(paying_back_on_time))*100,2)))



0              married
1              married
2              married
3              married
4    civil partnership
5    civil partnership
6              married
7              married
8    civil partnership
9              married
Name: family_status, dtype: object
family_status  : married 	 Default-Rate : 57.83
family_status  : civil partnership 	 Default-Rate : 19.12
family_status  : widow / widower 	 Default-Rate : 4.53
family_status  : divorced 	 Default-Rate : 5.64
family_status  : unmarried 	 Default-Rate : 12.87


**Conclusion**

[Write your conclusions based on your manipulations and observations.]

**Is there a correlation between income level and paying back on time?**

In [80]:
# Check the income level data and paying back on time

print(paying_back_on_time['income_type'].head(10))

# Calculating default-rate based on income level

for income_type, number in dict(Counter(paying_back_on_time['income_type'])).items():
    print("Income Type: {} \t Default-Rate : {}".format(income_type, round((number/len(paying_back_on_time))*100,2)))





0    employee
1    employee
2    employee
3    employee
4     retiree
5    business
6    business
7    employee
8    employee
9    employee
Name: income_type, dtype: object
Income Type: employee 	 Default-Rate : 50.83
Income Type: retiree 	 Default-Rate : 18.35
Income Type: business 	 Default-Rate : 23.84
Income Type: civil servant 	 Default-Rate : 6.96
Income Type: entrepreneur 	 Default-Rate : 0.01
Income Type: student 	 Default-Rate : 0.01
Income Type: unemployed 	 Default-Rate : 0.01


**Conclusion**

[Write your conclusions based on your manipulations and observations.]

**How does credit purpose affect the default rate?**

In [81]:
# Check the percentages for default rate for each credit purpose and analyze them
for purpose, number in dict(Counter(paying_back_on_time['purpose'])).items():
    print("Purpose: {} \t Default-Rate : {}".format(purpose, round((number/len(paying_back_on_time))*100,2)))



Purpose: purchase of the house 	 Default-Rate : 3.02
Purpose: car purchase 	 Default-Rate : 2.13
Purpose: supplementary education 	 Default-Rate : 2.13
Purpose: to have a wedding 	 Default-Rate : 3.58
Purpose: housing transactions 	 Default-Rate : 3.06
Purpose: education 	 Default-Rate : 2.09
Purpose: having a wedding 	 Default-Rate : 3.58
Purpose: purchase of the house for my family 	 Default-Rate : 3.02
Purpose: buy real estate 	 Default-Rate : 2.92
Purpose: buy commercial real estate 	 Default-Rate : 3.12
Purpose: construction of own property 	 Default-Rate : 3.0
Purpose: property 	 Default-Rate : 3.01
Purpose: building a property 	 Default-Rate : 2.88
Purpose: buying a second-hand car 	 Default-Rate : 2.22
Purpose: buying my own car 	 Default-Rate : 2.34
Purpose: transactions with commercial real estate 	 Default-Rate : 3.03
Purpose: building a real estate 	 Default-Rate : 2.92
Purpose: housing 	 Default-Rate : 3.04
Purpose: transactions with my real estate 	 Default-Rate : 2.92
Pu

**Conclusion**

[Write your conclusions based on your manipulations and observations.]


# General Conclusion 

[List your conclusions in this final section. Make sure you include all your important conclusions you made that led you to the way you processed and analyzed the data. Cover the missing values, duplicates, and possible reasons and solutions for problematic artifacts that you had to address.]

[List your conclusions regarding the posed questions here as well.]
